NFL Project -> Extracted data from the espn website APIs. Use this github for api extraction:  Thank you by the way! Finding out what team_stats contribute most to winning. Definitely will answer more questions and give high-level interpretation of the data. Data is extracted for NFL season 2025 (regular season games only). Graphs and visualization will be implemented to clearly show the stats that contribute most to winning and maybe help NFL teams build better football teams for future seasons! Thanks for supporting and any further questions or suggestions would be greatly appreciated!

In [26]:
# Imports for dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [27]:
# Connecting to supabase database
from db import get_connection
from dotenv import load_dotenv
load_dotenv()
conn = get_connection()

In [28]:
#Getting team_stats table data where the games are already complete
query = """
SELECT * FROM team_stats WHERE win is NOT NULL 
"""
# Dataset stored in Pandas dataframe
df = pd.read_sql(query, conn)

In [29]:
#sanity checks
df.shape
df.head

<bound method NDFrame.head of        game_id  team_id  first_downs  total_plays  total_yards  ypp  \
0    401772510        9           22           56          307  5.5   
1    401772510       26           20           62          302  4.9   
2    401772714       16           18           58          347  6.0   
3    401772714       18           22           62          394  6.4   
4    401772830       30           16           56          260  4.6   
..         ...      ...          ...          ...          ...  ...   
539  401772958       22           22           56          457  8.2   
540  401772959       32           25           63          274  4.3   
541  401772959       26           18           62          307  5.0   
542  401772960        3           12           48          359  7.5   
543  401772960       27           24           73          390  5.3   

     tot_pass_yards  interceptions  sacks  tot_rush_yards  ...  \
0               188              0      0          

Learning about our NFL dataset. This NFL team stats database has 22 columns 2 of which will not be processed as stats, and they are team_id and game_id. The remaining 20 will be used as predictors for our model to predict the most important stats to winning. They are
1. first_downs (more first downs the more likely you are to score and thus win the game)
2. total_plays (more total plays a team makes the opportunities they have to score)
3. total_yards (the more offensive yards a team has the more likely they are to score)
4. ypp  ((yards per play)the more yards a team makes per play the more efficient their offense)
5. tot_pass_yards (does total passing yards lead to wins more than rushing yards?)
6. tot_rush_yards (see number 5)
7. interceptions (Are interceptions a more back-breaking turnover or fumbles?)
8. fumbles (see number 7)
9. sacks (a good metric to see if defenses really do win championships?(games in this case))
10. sack_yds_lost (see number 9)
Having an NFL coach and coordinators that provides a "winning" culture is important to winning football games? 
Now what does that exactly mean when we hear sports media analyst say it? Stats (11-15) study the important plays that define if a coach is preparing their team properly.   
11. third_down_conversion, third_down_attempts (these two metrics will be combined to one called 
third_down_rate to view a teams overall percentage rate on third down)
12. fourth_down_conversion, fourth_down_attempts (these two metrics will be combined to one called 
fourth_down_rate to view a teams overall percentage rate on fourth down. Another thing to note is going
for it on fourth down lead to football wins(does high conversion lead to victory, or is it best to be 
conservative and bring the special teams unit out))
13. red_zone_conversion, red_zone_attempts(a teams ability to score touchdowns when they are close to the endzone, again will consolidate to one stat)
14. penalities and penalty yds (how well is the team coached, lower penalties, generally better coached)
15. time_poss_seconds (convert to minutes for legibility at some point)

In [30]:
# Creating a map so the team name will display instead of the ID giving readers a much easier time 
# understanding the teams they are seeing in the dataset
team_map = {
    1 : "Arizona Cardinals",
    2 : "Atlanta Falcons",
    3 : "Baltimore Ravens",
    4 : "Buffalo Bills",
    5 : "Carolina Panthers",
    6 : "Chicago Bears",
    7 : "Cincinatti Bengals",
    8 : "Cleveland Browns",
    9 : "Dallas Cowboys",
    10 : "Denver Broncos",
    11 : "Detroit Lions",
    12 : "Green Bay Packers",
    13 : "Houston Texans",
    14: "Indianapolis Colts",
    15: "Jacksonville Jaguars",
    16: "Kansas City Chiefs",
    17: "Las Vegas Raiders",
    18: "Los Angeles Chargers",
    19: "Los Angeles Rams",
    20: "Miami Dolphins",
    21: "Minnesota Vikings",
    22: "New England Patriots",
    23: "New Orleans Saints",
    24: "New York Giants",
    25: "New York Jets",
    26: "Philadelphia Eagles",
    27: "Pittsburgh Steelers",
    28: "Seattle Seahawks",
    29: "San Francisco 49ers",
    30: "Tampa Bay Buccaneers",
    31: "Tennessee Titans",
    32: "Washington Commanders"
}

In [31]:
#Create a new team_name column to display answers to dataset questions more clearly
df['team_name'] = df['team_id'].map(team_map)

In [32]:
df['win'].mean() #win category not exactly 50 percent which is strange 0.498
df['interceptions'].max() #4
df.groupby('team_id')['interceptions'].max() #interceptions thrown by each team
max_pass_yds=df.groupby('team_id')['tot_pass_yards'].max() #show all 32 teams highest passing yards
m=max_pass_yds.max()
print(m)
df[df['tot_pass_yards'] == m][['team_id', 'tot_pass_yards']] #find the team_id with the most passing yards (Rams)




457


,team_id,tot_pass_yards
448,19,457
